In [4]:
%load_ext watermark
%watermark

11/02/2015 22:10:55

CPython 2.7.10
IPython 4.0.0

compiler   : GCC 4.4.7 20120313 (Red Hat 4.4.7-1)
system     : Linux
release    : 3.13.0-66-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit


**TWEETS HEATMAP OF MURCIA**

**1. Getting the tweets**

*In order to get the tweets, we need to use Twitter Streaming api. I found that [Tweepy](http://tweepy.readthedocs.org/en/latest/getting_started.html), one (of many) Twitter API Python wrappers does an outstanding job in capturing tweets via Twitter Streaming API*

Here is the code I used. I filtered the tweets by location. Twitter allows for filtering using a bounding box set of coordinates using the following structure:

**location=[sw_longitude, sw_latitude, ne_longitude, ne_latitude]**

*(interesting how some apis follow (lat, lon) and others (lon, lat). We need a standard)*

In [ ]:
import json
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener


ckey = YOUR_CONSUMER_KEY_HERE
csecret = YOUR_CONSUMER_SECRET_HERE
atoken = YOUR_TWITTER_APP_TOKEN_HERE
asecret = YOUR_TWITTER_APP_SECRET_HERE

murcia = [-1.157420, 37.951741, -1.081202, 38.029126] #Check it out, is a very nice city!

file =  open('tweets.txt', 'a')

class listener(StreamListener):

    def on_data(self, data):
        # Twitter returns data in JSON format - we need to decode it first
        try:
            decoded = json.loads(data)
        except Exception as e:
            print e #we don't want the listener to stop
            return True
   
        if decoded.get('geo') is not None:
            location = decoded.get('geo').get('coordinates')
        else:
            location = '[,]'
        text = decoded['text'].replace('\n',' ')
        user = '@' + decoded.get('user').get('screen_name')
        created = decoded.get('created_at')
        tweet = '%s|%s|%s|s\n' % (user,location,created,text)
        
        file.write(tweet)
        print tweet
        return True

    def on_error(self, status):
        print status

if __name__ == '__main__':
    print 'Starting'
    
    auth = OAuthHandler(ckey, csecret)
    auth.set_access_token(atoken, asecret)
    twitterStream = Stream(auth, listener())
    twitterStream.filter(locations=murcia)

**Run it...and wait.**

The script will capture all the tweets that fit within that bounding box we setup. 

One important thing to notice is that the api is not 100% accurate on the data it returns. I found several geocoded tweets that didn't belong to the specified box. 

Since the script has to be running in order to capture all the tweets, you can run this on a spare computer if you have one, or alternatively you can consider online services such as [RedHat](http://www.redhat.com/) or [PythonAnywhere](https://www.pythonanywhere.com/), or rent your ownn tiny machine on the cloud with services like [Digital Ocean](digitalocean.com) or [Amazon Web Services](aws.amazon.com)

<hr>